In [3]:
import pandas as pd

In [4]:
# df16 = pd.read_csv("finbraRREO2016.csv", encoding='latin-1', skiprows=5, sep=';')
# df17 = pd.read_csv("finbraRREO2017.csv", encoding='latin-1', skiprows=5, sep=';')
# df18 = pd.read_csv("finbraRREO2018.csv", encoding='latin-1', skiprows=5, sep=';')
# df19 = pd.read_csv("finbraRREO2019.csv", encoding='latin-1', skiprows=5, sep=';')
# df20 = pd.read_csv("finbraRREO2020.csv", encoding='latin-1', skiprows=5, sep=';')
# df21 = pd.read_csv("finbraRREO2021.csv", encoding='latin-1', skiprows=5, sep=';')
# df22 = pd.read_csv("finbraRREO2022.csv", encoding='latin-1', skiprows=5, sep=';')
# df23 = pd.read_csv("finbraRREO2023.csv", encoding='latin-1', skiprows=5, sep=';')

# df16 = df16[(df16['UF'].isin(['PB', 'PE', 'AL', 'BA', 'RJ'])) & (df16['Conta'].isin(['ISS', 'IPTU', 'ITBI']))]
# df17 = df17[(df17['UF'].isin(['PB', 'PE', 'AL', 'BA', 'RJ'])) & (df17['Conta'].isin(['ISS', 'IPTU', 'ITBI']))]
# df18 = df18[(df18['UF'].isin(['PB', 'PE', 'AL', 'BA', 'RJ'])) & (df18['Conta'].isin(['ISS', 'IPTU', 'ITBI']))]
# df19 = df19[(df19['UF'].isin(['PB', 'PE', 'AL', 'BA', 'RJ'])) & (df19['Conta'].isin(['ISS', 'IPTU', 'ITBI']))]
# df20 = df20[(df20['UF'].isin(['PB', 'PE', 'AL', 'BA', 'RJ'])) & (df20['Conta'].isin(['ISS', 'IPTU', 'ITBI']))]
# df21 = df21[(df21['UF'].isin(['PB', 'PE', 'AL', 'BA', 'RJ'])) & (df21['Conta'].isin(['ISS', 'IPTU', 'ITBI']))]
# df22 = df22[(df22['UF'].isin(['PB', 'PE', 'AL', 'BA', 'RJ'])) & (df22['Conta'].isin(['ISS', 'IPTU', 'ITBI']))]
# df23 = df23[(df23['UF'].isin(['PB', 'PE', 'AL', 'BA', 'RJ'])) & (df23['Conta'].isin(['ISS', 'IPTU', 'ITBI']))]


# df16['Ano'] = 2016
# df17['Ano'] = 2017
# df18['Ano'] = 2018
# df19['Ano'] = 2019
# df20['Ano'] = 2020
# df21['Ano'] = 2021
# df22['Ano'] = 2022
# df23['Ano'] = 2023

In [5]:
# df = pd.concat([df16, df17, df18, df19, df20, df21, df22, df23])
# df.to_csv('finbraRREOgeral.csv', index=False)

In [6]:
df = pd.read_csv('finbraRREOgeral.csv')

In [8]:
df = df[df['Conta'].isin(['ISS', 'IPTU', 'ITBI'])]

In [9]:
df

,Instituição,Cod.IBGE,UF,População,Coluna,Conta,Identificador da Conta,Valor,Ano
0,Prefeitura Municipal de Angelim - PE,2601003,PE,10882,<MR-11>,IPTU,siconfi-cor_IPTULiquidoExcetoTransferenciasEFU...,"1397,71",2016
1,Prefeitura Municipal de Angelim - PE,2601003,PE,10882,<MR-11>,ISS,siconfi-cor_ISSLiquidoExcetoTransferenciasEFUNDEB,"40787,32",2016
2,Prefeitura Municipal de Angelim - PE,2601003,PE,10882,<MR-11>,ITBI,siconfi-cor_ITBILiquidoExcetoTransferenciasEFU...,"2760,00",2016
3,Prefeitura Municipal de Angelim - PE,2601003,PE,10882,<MR-10>,IPTU,siconfi-cor_IPTULiquidoExcetoTransferenciasEFU...,"916,27",2016
4,Prefeitura Municipal de Angelim - PE,2601003,PE,10882,<MR-10>,ISS,siconfi-cor_ISSLiquidoExcetoTransferenciasEFUNDEB,"33771,79",2016
...,...,...,...,...,...,...,...,...,...
278970,Prefeitura Municipal de Miracema - RJ,3303005,RJ,27134,TOTAL (ÚLTIMOS 12 MESES),ISS,siconfi-cor_ISSLiquidoExcetoTransferenciasEFUNDEB,"613926,98",2023
278971,Prefeitura Municipal de Miracema - RJ,3303005,RJ,27134,TOTAL (ÚLTIMOS 12 MESES),ITBI,siconfi-cor_ITBILiquidoExcetoTransferenciasEFU...,"61975,71",2023
278972,Prefeitura Municipal de Miracema - RJ,3303005,RJ,27134,PREVISÃO ATUALIZADA 2023,IPTU,siconfi-cor_IPTULiquidoExcetoTransferenciasEFU...,"3950000,00",2023
278973,Prefeitura Municipal de Miracema - RJ,3303005,RJ,27134,PREVISÃO ATUALIZADA 2023,ISS,siconfi-cor_ISSLiquidoExcetoTransferenciasEFUNDEB,"6300000,00",2023
